# Overview map

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import sys, os
sys.path.append(os.getcwd() + '/..')
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, maps, open_tar_shapefile, open_zip_shapefile
import os
import matplotlib.patches as mpatches
from oggm import utils

import seaborn as sns
import cartopy
import cartopy.crs as ccrs
from matplotlib.image import imread

In [ ]:
# go down from rgi7_scripts/workflow
data_dir = '../../../rgi7_data/'

l4_dir = os.path.join(data_dir, 'l4_rgi7b0_tar')

In [ ]:
rgi = []
for reg in range(1, 20):
    shp = gpd.read_file(utils.get_rgi_region_file(f'{reg:02d}', version='60'))
    shp['area'] = shp.to_crs({'proj':'cea'}).area
    shp['O1Region'] = f'{reg:02d}'
    rgi.append(shp)
rgi6 = pd.concat(rgi)
rgi6 = rgi6.loc[rgi6['Connect'] != 2]
rgi6['area'] = rgi6['area'] * 1e-6

In [ ]:
reg_file = open_zip_shapefile(os.path.join(data_dir, '00_rgi70_regions/00_rgi70_O1Regions.zip'))
sreg_file = open_zip_shapefile(os.path.join(data_dir, '00_rgi70_regions/00_rgi70_O2Regions.zip'))

In [ ]:
for n, s in zip(reg_file.o1region, reg_file.full_name):
    print(f'{n} {s}')

In [ ]:
rgi = []
for reg in range(1, 20):
    shp = open_tar_shapefile(os.path.join(l4_dir, 'RGI2000-v7.0-G-' + reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0].long_code + '.tar.gz'))
    rgi.append(shp)
rgi7 = pd.concat(rgi)

In [ ]:
not_valid6 = ~ rgi6.is_valid
not_valid7 = ~ rgi7.is_valid

In [ ]:
not_valid6.sum(), not_valid7.sum()

In [ ]:
not_valid6.sum() / len(rgi6)

In [ ]:
rgi6['Area'].sum(), rgi6['area'].sum(), rgi7['area_km2'].sum()

In [ ]:
df = rgi6.groupby('O1Region')[['area']].sum()
df.columns = ['Area RGI6']

In [ ]:
df['Area RGI7'] = rgi7.groupby('o1region')[['area_km2']].sum()

In [ ]:
df['Diff A (%)'] = (df['Area RGI7'] / df['Area RGI6'] - 1) * 100

In [ ]:
df['N RGI6'] = rgi6.groupby('O1Region').count()['area']
df['N RGI7'] = rgi7.groupby('o1region').count()['area_km2']

df['Diff N (%)'] = (df['N RGI7'] / df['N RGI6'] - 1) * 100

In [ ]:
ss = df.sum()
ss.name = 'Global'
df = pd.concat([df, ss.to_frame().T])

df['Diff A (%)'] = (df['Area RGI7'] / df['Area RGI6'] - 1) * 100
df['Diff N (%)'] = (df['N RGI7'] / df['N RGI6'] - 1) * 100

In [ ]:
df['Area RGI6'] = df['Area RGI6'].astype(float).round(0).astype(int) 
df['Area RGI7'] = df['Area RGI7'].astype(float).round(0).astype(int) 
df['N RGI6'] = df['N RGI6'].astype(int) 
df['N RGI7'] = df['N RGI7'].astype(int) 

df['Diff A (%)'] = df['Diff A (%)'].round(1).replace(-0, 0)
df['Diff N (%)'] = df['Diff N (%)'].round(1).replace(-0, 0)

In [ ]:
print(f"New in N: {len(rgi7)-len(rgi6)}")

In [ ]:
df

In [ ]:
s1 = rgi7.loc[rgi7.is_rgi6 == 1]
print(f"Same as RGI6: N={len(s1)}, A={int(s1['area_km2'].sum())} km²")

s2 = rgi7.loc[rgi7.is_rgi6 == 0]
print(f"New  in RGI7: N={len(s2)}, A={int(s2['area_km2'].sum())} km²")

print(f"New in % area: {s2['area_km2'].sum() / rgi7['area_km2'].sum()}")

print(f"New in % number: {len(s2) / len(rgi7)}")

In [ ]:
df.index.name = 'Region'
df_formd = df.copy()

In [ ]:
df_formd.index = [f'[](regions/rgi{i}.md)' for i in df_formd.index[:-1]] + ['Global']

In [ ]:
print(df_formd.to_markdown())

In [ ]:
df['Full name'] = list(reg_file.full_name.unique()) + ['']
df['Code'] = ['`' + r + '`' for r in reg_file.long_code.unique()] + ['']

In [ ]:
for_print = df[['Full name', 'Code', 'N RGI7', 'Area RGI7']]

In [ ]:
for_print.columns = ['Full name', 'Code', 'Glaciers', 'Area (km²)']

In [ ]:
print(for_print.to_markdown())

In [ ]:
rgi7.primeclass.value_counts().sort_index(), len(rgi7)

In [ ]:
sreg_file = sreg_file.drop_duplicates('o2region')
sreg_file

In [ ]:
dfsr = sreg_file[['o2region', 'full_name', 'long_code']].copy().set_index('o2region')

dfsr.columns = ['Full name', 'Code']

dfsr['Glaciers'] = rgi7.groupby('o2region')['cenlon'].count()
dfsr['Area (km²)'] = rgi7.groupby('o2region')['area_km2'].sum()

ss = dfsr.sum()
ss.name = 'Global'
dfsr = pd.concat([dfsr, ss.to_frame().T])

dfsr.loc['Global', 'Full name'] = ''
dfsr.loc['Global', 'Code'] = ''

dfsr['Area (km²)']  = dfsr['Area (km²)'].astype(float).round(0).fillna(0).astype(int) 
dfsr['Glaciers']  = dfsr['Glaciers'].fillna(0).astype(int) 

dfsr['Code'] = ['`' + r + '`' for r in dfsr.Code]

In [ ]:
print(dfsr.to_markdown())

In [ ]:
import salem
df = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions', '00_rgi70_O1Regions.shp'))

In [ ]:
%config InlineBackend.figure_format = 'retina'
%alias convert convert

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])

handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7.loc[rgi7.is_rgi6 == 1]
ax.scatter(s.cenlon.values, s.cenlat.values, color='C0', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C0', label=f"Same as RGI6"))  #  - N={len(s)}, A={int(s['area'].sum()*1e-6)} km²

s = rgi7.loc[rgi7.is_rgi6 == 0]
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"New in RGI7"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map_wrgi6.png', dpi=300, bbox_inches='tight')

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map_wrgi6.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_wrgi6_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"RGI7 glacier outlines"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map.png', dpi=300, bbox_inches='tight')

# ax.scatter(dfr.lon.values, dfr.lat.values, edgecolor='k', facecolor='tab:cyan', s=24)

# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);
# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())

# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])

handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi6
ax.scatter(s.CenLon.values, s.CenLat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"RGI6 glacier outlines"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map_rgi6.png', dpi=300, bbox_inches='tight')

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map_rgi6.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_rgi6_small.jpeg

In [ ]:
adf = pd.DataFrame()
adf.loc['< 1 km²', 'RGI6 (N)'] = (rgi6['area'] < 1).sum()
adf.loc['< 1 km²', 'RGI6 (%)'] = (rgi6['area'] < 1).sum() / len(rgi6) * 100
adf.loc['< 1 km²', 'RGI7 (N)'] = (rgi7['area_km2'] < 1).sum()
adf.loc['< 1 km²', 'RGI7 (%)'] = (rgi7['area_km2'] < 1).sum() / len(rgi7) * 100
adf.loc['1-10 km²', 'RGI6 (N)'] = ((rgi6['area'] >= 1) & (rgi6['area'] < 10)).sum()
adf.loc['1-10 km²', 'RGI6 (%)'] = ((rgi6['area'] >= 1) & (rgi6['area'] < 10)).sum() / len(rgi6) * 100
adf.loc['1-10 km²', 'RGI7 (N)'] = ((rgi7['area_km2'] >= 1) & (rgi7['area_km2'] < 10)).sum()
adf.loc['1-10 km²', 'RGI7 (%)'] = ((rgi7['area_km2'] >= 1) & (rgi7['area_km2'] < 10)).sum() / len(rgi7) * 100
adf.loc['10-100 km²', 'RGI6 (N)'] = ((rgi6['area'] >= 10) & (rgi6['area'] < 100)).sum()
adf.loc['10-100 km²', 'RGI6 (%)'] = ((rgi6['area'] >= 10) & (rgi6['area'] < 100)).sum() / len(rgi6) * 100
adf.loc['10-100 km²', 'RGI7 (N)'] = ((rgi7['area_km2'] >= 10) & (rgi7['area_km2'] < 100)).sum()
adf.loc['10-100 km²', 'RGI7 (%)'] = ((rgi7['area_km2'] >= 10) & (rgi7['area_km2'] < 100)).sum() / len(rgi7) * 100
adf.loc['> 100 km²', 'RGI6 (N)'] = (rgi6['area'] >= 100).sum()
adf.loc['> 100 km²', 'RGI6 (%)'] = (rgi6['area'] >= 100).sum() / len(rgi6) * 100
adf.loc['> 100 km²', 'RGI7 (N)'] = (rgi7['area_km2'] >= 100).sum()
adf.loc['> 100 km²', 'RGI7 (%)'] = (rgi7['area_km2'] >= 100).sum() / len(rgi7) * 100
adf.loc['Total'] = adf.sum()
adf[['RGI6 (N)', 'RGI7 (N)']] = adf[['RGI6 (N)', 'RGI7 (N)']].astype(int)
adf[['RGI6 (%)', 'RGI7 (%)']] = adf[['RGI6 (%)', 'RGI7 (%)']].round(2)
adf

In [ ]:
print(adf.to_markdown())

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('ticks'):

    bins = np.logspace(-2, 3, 100)

    h7, b7 = np.histogram(rgi7['area_km2'], bins=bins)
    h6, b6 = np.histogram(rgi6['area'], bins=bins)

    f, ax = plt.subplots(figsize=(12, 7))
    ax.plot(b6[:-1], h6, label='RGI6');
    ax.plot(b7[:-1], h7, label='RGI7');
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.legend()
    ax.set_title('Number of glaciers per size category (global)')
    ax.set_xlabel('Glacier area (km², logscale)');
    ax.set_ylabel('Glacier number (logscale)');
    plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_histogram.png', dpi=300, bbox_inches='tight')

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('ticks'):
    
    bins = np.logspace(-2, 2, 100)
    reg = ['13', '14', '15']
    reg = ['17']

    h7, b7 = np.histogram(rgi7.loc[rgi7.o1region.isin(reg)]['area_km2'], bins=bins)
    h6, b6 = np.histogram(rgi6.loc[rgi6.O1Region.isin(reg)]['area'], bins=bins)

    f, ax = plt.subplots(figsize=(12, 7))
    ax.plot(b6[:-1], h6, label='RGI6');
    ax.plot(b7[:-1], h7, label='RGI7');
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.legend()
    ax.set_title(f'Number of glaciers per size category (region {reg})')
    ax.set_xlabel('Glacier area (km², logscale)');
    ax.set_ylabel('Glacier number (logscale)');

In [ ]:
rgi7['year'] = [int(y.split('-')[0]) for y in rgi7['src_date']]

In [ ]:
rgi6['year'] = [int(y[0:4]) for y in rgi6['BgnDate']]
rgi6['year'].unique()

In [ ]:
(rgi6['year'] == -999).sum()

In [ ]:
dy6 = np.abs(rgi6['year'] - 2000)
dy7 = np.abs(rgi7['year'] - 2000)

In [ ]:
ydf = pd.DataFrame()
ydf.loc['2000 ± 2 years', 'RGI6 (%)'] = (dy6 <= 2).sum() / len(rgi6) * 100
ydf.loc['2000 ± 2 years', 'RGI7 (%)'] = (dy7 <= 2).sum() / len(rgi7) * 100
ydf.loc['2000 ± 2-5 years', 'RGI6 (%)'] = ((dy6 <= 5) & (dy6 > 2)).sum() / len(rgi6) * 100
ydf.loc['2000 ± 2-5 years', 'RGI7 (%)'] = ((dy7 <= 5) & (dy7 > 2)).sum() / len(rgi7) * 100
ydf.loc['2000 ± 5-10 years', 'RGI6 (%)'] = ((dy6 <= 10) & (dy6 > 5)).sum() / len(rgi6) * 100
ydf.loc['2000 ± 5-10 years', 'RGI7 (%)'] = ((dy7 <= 10) & (dy7 > 5)).sum() / len(rgi7) * 100
ydf.loc['> 2000 ± 10 years', 'RGI6 (%)'] = (dy6 > 10).sum() / len(rgi6) * 100
ydf.loc['> 2000 ± 10 years', 'RGI7 (%)'] = (dy7 > 10).sum() / len(rgi7) * 100
ydf = ydf.round(1)
ydf

In [ ]:
print(ydf.to_markdown())

In [ ]:
len(rgi7) - len(rgi7.anlys_id.unique())

In [ ]:
fac = 1.3
f = plt.figure(figsize=(7*fac, 7*fac))
ax = plt.axes(projection=ccrs.Orthographic(-20, 45))

fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90]);

s = rgi7
handles = []
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=0.1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"Glacier outlines"))
ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();